In [1]:
import pymongo
from pymongo import MongoClient
import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster
import pandas as pd
from pandas import DataFrame
from dotenv import load_dotenv
import os
import requests
import json
from functools import reduce
import operator
import geopandas as gpd
import cartoframes
from cartoframes.viz import Map, Layer, popup_element

In [2]:
client = MongoClient()
db = client.companies
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'companies')

### Based on the information provided, we are going to select a possible location for the office based on the available information.

The applied filter assums a **number of emplooyees of 90**. 

The reason? Currently, a huge number of companies have implanted a ***work from home policy*** but, additionally the company ***expect to increase their employees number in the future***, so being continously changing the location would be counterproductive.

Additionaly, we have selected **New York as the City** to locate the office as it is the USA city with more **Starbucks**.

In [3]:
filt1 = {"number_of_employees": {"$eq":90},"offices.city":{"$ne": "",}}
project1 = {"_id":0,"offices.city": 1,"number_of_employees": 1}
results1 = db.offices.find(filt1,project1).sort("number_of_employees",1)#.skip(1).limit(1)
results1

In [4]:
pd.DataFrame.from_dict(results1)

,number_of_employees,offices
0,90,{'city': 'Amsterdam'}
1,90,{'city': 'San Francisco'}
2,90,{'city': 'Singapore'}
3,90,{'city': 'Denver'}
4,90,{'city': 'San Francisco'}
5,90,{'city': 'Palo Alto'}
6,90,{'city': 'Chicago'}
7,90,{'city': 'San Francisco'}
8,90,{'city': 'Irving'}
9,90,{'city': 'South San Francisco'}


Additionaly, we have selected **New York as the City** to locate the office as it is the USA city with more **Starbucks**.

In [5]:
filt1 = {"number_of_employees": {"$eq":90},"offices.city": {'$eq': "New York"}}
project1 = {"_id":0,"name":1,"offices.city": 1,"number_of_employees": 1,"offices.latitude":1,"offices.longitude":1}
results1 = db.offices.find(filt1,project1).sort("number_of_employees",1)#.skip(1).limit(1)
results1

In [6]:
office_location = list(results1)
office_location

[{'name': 'Aleri',
  'number_of_employees': 90,
  'offices': {'city': 'New York',
   'latitude': 40.7592189,
   'longitude': -73.9783534}}]

In [7]:
pd.DataFrame.from_dict(office_location)

,name,number_of_employees,offices
0,Aleri,90,"{'city': 'New York', 'latitude': 40.7592189, '..."


## OFFICE LOCATION

In [8]:
office_lat = 40.7592189
office_long = -73.9783534

In [9]:
map_1 = folium.Map(location = [office_lat, office_long], zoom_start = 16)
map_1

In [10]:
office_loc = folium.Marker(location = [office_lat, office_long], tooltip = "Office location proposal")
# tooltip nos sirve para crear texto en los marcadores
office_loc.add_to(map_1)
map_1

icono = Icon(color = "blue",
             prefix = "fa",
             icon = "building-o",
             icon_color = "black",
             tooltip = "Office location proposal")

In [11]:
office_location = [office_lat, office_long]
marker_office = Marker(location = office_location, icon = icono)
marker_office.add_to(map_1)
map_1

In [12]:
load_dotenv()

True

In [13]:
city = "New York"
def geocode(address):
    data = requests.get(f"https://geocode.xyz/{address}?json=1").json()
    try:
        return {
            "type":"Point",
            "coordinates":[float(data["longt"]),float(data["latt"])]}
    except:
        return data

In [14]:
donde = "New York"
data = requests.get(f"https://geocode.xyz/{donde}?json=1").json()

In [15]:
print(data)

{'standard': {'addresst': {}, 'city': 'New York', 'prov': 'US', 'countryname': 'United States of America', 'postal': {}, 'confidence': '0.90'}, 'longt': '-73.95861', 'alt': {'loc': {'longt': '-73.9536869565217', 'prov': 'NY', 'city': 'NEW YORK', 'postal': '10075', 'score': '25', 'latt': '40.7724386956522'}}, 'elevation': {}, 'latt': '40.68908'}


In [16]:
nyc = geocode(city)

In [17]:
nyc

{'type': 'Point', 'coordinates': [-73.95861, 40.68908]}

In [18]:
nyc = {'type': 'Point', 'coordinates': [40.7592189, -73.9783534]}

In [19]:
tok1 = os.getenv("tok1")
tok2 = os.getenv("tok2")

### Búsqueda de Starbucks

In [20]:
url_query = 'https://api.foursquare.com/v2/venues/explore'
starbucks = "556f676fbd6a75a99038d8ec"

In [21]:
parametros = {"client_id" : tok1,
              "client_secret" : tok2,
              "v": "20180323",
              "ll": f"{nyc.get('coordinates')[0]},{nyc.get('coordinates')[1]}",
              "query":f"Starbucks",
              "radius":500
}

In [22]:
resp = requests.get(url= url_query, params=parametros)
data = json.loads(resp.text)

In [23]:
data.keys()

dict_keys(['meta', 'response'])

In [24]:
decoding_data = data.get("response")

In [25]:
for k,v in decoding_data.items():
    print(k,v,"\n")

suggestedFilters {'header': 'Tap to show:', 'filters': [{'name': 'Open now', 'key': 'openNow'}, {'name': '$-$$$$', 'key': 'price'}]} 

headerLocation Rockefeller Center 

headerFullLocation Rockefeller Center, New York 

headerLocationGranularity neighborhood 

query starbucks 

totalResults 20 

suggestedBounds {'ne': {'lat': 40.7637189045, 'lng': -73.97242357822688}, 'sw': {'lat': 40.7547188955, 'lng': -73.98428322177313}} 

groups [{'type': 'Recommended Places', 'name': 'recommended', 'items': [{'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '58c8c7ca9900e624af24a199', 'name': 'Starbucks', 'contact': {}, 'location': {'address': '28 W 48th St', 'crossStreet': 'btwen 5th & 6th Ave', 'lat': 40.75814852475969, 'lng': -73.97941981951507, 'labeledLatLngs': [{'label': 'display', 'lat': 40.75814852475969, 'lng': -73.97941981951507}], 'distance': 149, 'postalCode': '10036', 'cc': 'US', 'city

In [26]:
decoded = decoding_data.get("groups")[0]

In [27]:
starbucks = decoded.get("items")

In [28]:
for k,v in starbucks[0].items():
    print(k,"-->",v,"\n")

reasons --> {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]} 

venue --> {'id': '58c8c7ca9900e624af24a199', 'name': 'Starbucks', 'contact': {}, 'location': {'address': '28 W 48th St', 'crossStreet': 'btwen 5th & 6th Ave', 'lat': 40.75814852475969, 'lng': -73.97941981951507, 'labeledLatLngs': [{'label': 'display', 'lat': 40.75814852475969, 'lng': -73.97941981951507}], 'distance': 149, 'postalCode': '10036', 'cc': 'US', 'city': 'New York', 'state': 'NY', 'country': 'United States', 'formattedAddress': ['28 W 48th St (btwen 5th & 6th Ave)', 'New York, NY 10036', 'United States']}, 'categories': [{'id': '4bf58dd8d48988d1e0931735', 'name': 'Coffee Shop', 'pluralName': 'Coffee Shops', 'shortName': 'Coffee Shop', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_', 'suffix': '.png'}, 'primary': True}], 'verified': True, 'stats': {'tipCount': 0, 'usersCount': 0, 'checkinsCount': 0, 'visitsCount': 0

In [29]:
map_starbucks = ["venue","name"]
m_latitud = ["venue","location","lat"]
m_longitud = ["venue","location","lng"]

In [30]:
def getFromDict(diccionario,mapa):
    return reduce (operator.getitem,mapa,diccionario)

In [31]:
print(getFromDict(starbucks[0],map_starbucks))

Starbucks


In [32]:
starbucks_nyc = []
for dic in starbucks:
    paralista = {}
    paralista["name"] = getFromDict(dic,map_starbucks)
    paralista["latitud"] = getFromDict(dic,m_latitud)
    paralista["longitud"] = getFromDict(dic,m_longitud)
    starbucks_nyc.append(paralista)

In [33]:
starbucks_nyc[0]

{'name': 'Starbucks',
 'latitud': 40.75814852475969,
 'longitud': -73.97941981951507}

In [34]:
dataframefinal = pd.DataFrame(starbucks_nyc)
dataframefinal.head()

,name,latitud,longitud
0,Starbucks,40.758149,-73.979420
1,Starbucks,40.759535,-73.980981
2,Starbucks,40.760602,-73.976467
3,Starbucks,40.761845,-73.978700
4,Starbucks,40.756665,-73.974397


In [35]:
gdf = gpd.GeoDataFrame(dataframefinal, geometry = gpd.points_from_xy(dataframefinal.longitud,dataframefinal.latitud))
gdf.head()

,name,latitud,longitud,geometry
0,Starbucks,40.758149,-73.979420,POINT (-73.97942 40.75815)
1,Starbucks,40.759535,-73.980981,POINT (-73.98098 40.75953)
2,Starbucks,40.760602,-73.976467,POINT (-73.97647 40.76060)
3,Starbucks,40.761845,-73.978700,POINT (-73.97870 40.76185)
4,Starbucks,40.756665,-73.974397,POINT (-73.97440 40.75666)


In [36]:
#!pip3 install cartoframes

In [37]:
Map(Layer(gdf, popup_hover = [popup_element("name","Starbucks in NYC")]))

### Looking for vegan restaurants

In [38]:
url2 = 'https://api.foursquare.com/v2/venues/explore'
vegan = "4bf58dd8d48988d1d3941735"

In [39]:
parametros_vegan = {"client_id" : tok1,
              "client_secret" : tok2,
              "v": "20180323",
              "ll": f"{nyc.get('coordinates')[0]},{nyc.get('coordinates')[1]}",
              "query":f"vegans",
                "radius":500
}

In [40]:
resp2 = requests.get(url = url_query, params = parametros_vegan)
data2 = json.loads(resp2.text)

In [41]:
data2.keys()

dict_keys(['meta', 'response'])

In [42]:
decoding_data2 = data2.get("response")

In [43]:
for k,v in decoding_data2.items():
    print(k,v,"\n")

suggestedFilters {'header': 'Tap to show:', 'filters': [{'name': '$-$$$$', 'key': 'price'}, {'name': 'Open now', 'key': 'openNow'}]} 

headerLocation Rockefeller Center 

headerFullLocation Rockefeller Center, New York 

headerLocationGranularity neighborhood 

query vegans 

totalResults 33 

suggestedBounds {'ne': {'lat': 40.7637189045, 'lng': -73.97242357822688}, 'sw': {'lat': 40.7547188955, 'lng': -73.98428322177313}} 

groups [{'type': 'Recommended Places', 'name': 'recommended', 'items': [{'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '545cf6be498e3a7b3a1af473', 'name': 'Beyond Sushi', 'contact': {}, 'location': {'address': '62 W 56th St', 'crossStreet': '6th Avenue', 'lat': 40.76345369164551, 'lng': -73.9771627466244, 'labeledLatLngs': [{'label': 'display', 'lat': 40.76345369164551, 'lng': -73.9771627466244}, {'label': 'entrance', 'lat': 40.763325, 'lng': -73.977164}], 'distanc

In [44]:
decoded2 = decoding_data2.get("groups")[0]

In [45]:
vegan = decoded2.get("items")

In [46]:
for k,v in vegan[0].items():
    print(k,"-->",v,"\n")

reasons --> {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]} 

venue --> {'id': '545cf6be498e3a7b3a1af473', 'name': 'Beyond Sushi', 'contact': {}, 'location': {'address': '62 W 56th St', 'crossStreet': '6th Avenue', 'lat': 40.76345369164551, 'lng': -73.9771627466244, 'labeledLatLngs': [{'label': 'display', 'lat': 40.76345369164551, 'lng': -73.9771627466244}, {'label': 'entrance', 'lat': 40.763325, 'lng': -73.977164}], 'distance': 481, 'postalCode': '10019', 'cc': 'US', 'city': 'New York', 'state': 'NY', 'country': 'United States', 'formattedAddress': ['62 W 56th St (6th Avenue)', 'New York, NY 10019', 'United States']}, 'categories': [{'id': '4bf58dd8d48988d1d3941735', 'name': 'Vegetarian / Vegan Restaurant', 'pluralName': 'Vegetarian / Vegan Restaurants', 'shortName': 'Vegetarian / Vegan', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/vegetarian_', 'suffix': '.png'}, 'primary': True}], 'verified'

In [47]:
map_vegan = ["venue","name"]
m_latitudvegan = ["venue","location","lat"]
m_longitudvegan = ["venue","location","lng"]

In [48]:
def getFromDict(diccionario2,mapa2):
    return reduce (operator.getitem,mapa2,diccionario2)

In [49]:
print(getFromDict(vegan[0],map_vegan))

Beyond Sushi


In [50]:
vegan_nyc = []
for dic in vegan:
    paralista2 = {}
    paralista2["name"] = getFromDict(dic,map_vegan)
    paralista2["latitud"] = getFromDict(dic,m_latitudvegan)
    paralista2["longitud"] = getFromDict(dic,m_longitudvegan)
    vegan_nyc.append(paralista2)

In [51]:
vegan_nyc[0]

{'name': 'Beyond Sushi',
 'latitud': 40.76345369164551,
 'longitud': -73.9771627466244}

In [52]:
dataframefina2 = pd.DataFrame(vegan_nyc)
dataframefina2.head()

,name,latitud,longitud
0,Beyond Sushi,40.763454,-73.977163
1,by CHLOE.,40.758063,-73.978854
2,Van Leeuwen Ice Cream,40.759578,-73.980338
3,Taco Dumbo,40.760295,-73.977430
4,L’Avenue,40.758756,-73.977673


In [53]:
gdf2 = gpd.GeoDataFrame(dataframefina2, geometry = gpd.points_from_xy(dataframefina2.longitud,dataframefina2.latitud))
gdf2.head()

,name,latitud,longitud,geometry
0,Beyond Sushi,40.763454,-73.977163,POINT (-73.97716 40.76345)
1,by CHLOE.,40.758063,-73.978854,POINT (-73.97885 40.75806)
2,Van Leeuwen Ice Cream,40.759578,-73.980338,POINT (-73.98034 40.75958)
3,Taco Dumbo,40.760295,-73.977430,POINT (-73.97743 40.76029)
4,L’Avenue,40.758756,-73.977673,POINT (-73.97767 40.75876)


In [54]:
Map(Layer(gdf2, popup_hover = [popup_element("name","Vegan restaurants in NYC")]))

### 3. Looking for a basketball stadium


In [55]:
url3 = 'https://api.foursquare.com/v2/venues/explore'
basketball_stadium = "4bf58dd8d48988d18b941735"
# Ubicado al mens a 10 km

In [56]:
parametros_basket = {"client_id" : tok1,
              "client_secret" : tok2,
              "v": "20180323",
              "ll": f"{nyc.get('coordinates')[0]},{nyc.get('coordinates')[1]}",
              "query":f"basket_stadium",
                "radius":1000
}

In [57]:
resp3 = requests.get(url = url_query, params = parametros_basket)
data3 = json.loads(resp3.text)

In [58]:
data3.keys()

dict_keys(['meta', 'response'])

In [59]:
decoding_data3 = data3.get("response")
for k,v in decoding_data3.items():
    print(k,v,"\n")

headerLocation Rockefeller Center 

headerFullLocation Rockefeller Center, New York 

headerLocationGranularity neighborhood 

query basket_stadium 

totalResults 4 

suggestedBounds {'ne': {'lat': 40.76821890900001, 'lng': -73.96649375645376}, 'sw': {'lat': 40.75021889099999, 'lng': -73.99021304354625}} 

groups [{'type': 'Recommended Places', 'name': 'recommended', 'items': [{'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '451d2920f964a5208d3a1fe3', 'name': 'Regal E-Walk 4DX & RPX', 'contact': {}, 'location': {'address': '247 W. 42nd St.', 'crossStreet': 'between 7th and 8th Avenue', 'lat': 40.756960133814175, 'lng': -73.9892856652269, 'labeledLatLngs': [{'label': 'display', 'lat': 40.756960133814175, 'lng': -73.9892856652269}], 'distance': 955, 'postalCode': '10036', 'cc': 'US', 'city': 'New York', 'state': 'NY', 'country': 'United States', 'formattedAddress': ['247 W. 42nd St. (bet

In [60]:
decoded3 = decoding_data3.get("groups")[0]

In [61]:
basket_stadium = decoded3.get("items")

In [62]:
map_stadium = ["venue","name"]

In [63]:
m_latitud_stadium = ["venue","location","lat"]
m_longitud_stadium = ["venue","location","lng"]
def getFromDict(diccionario3,mapa3):
    return reduce (operator.getitem,mapa3,diccionario3)

In [64]:
print(getFromDict(basket_stadium[0],map_stadium))

Regal E-Walk 4DX & RPX


In [65]:
basket_stadium_nyc = []
for dic in basket_stadium:
    paralista3 = {}
    paralista3["name"] = getFromDict(dic,map_stadium)
    paralista3["latitud"] = getFromDict(dic,m_latitud_stadium)
    paralista3["longitud"] = getFromDict(dic,m_longitud_stadium)
    basket_stadium_nyc.append(paralista3)

In [66]:
basket_stadium_nyc[0]
dataframefina3 = pd.DataFrame(basket_stadium_nyc)
dataframefina3.head()

,name,latitud,longitud
0,Regal E-Walk 4DX & RPX,40.756960,-73.989286
1,Zaro's Bakery,40.752875,-73.976946
2,Boomer Esiason's Stadium Grill,40.757840,-73.987099
3,Stadium Grill At Bowlmor Lanes,40.757211,-73.987395


In [67]:
gdf3 = gpd.GeoDataFrame(dataframefina3, geometry = gpd.points_from_xy(dataframefina3.longitud,dataframefina3.latitud))
gdf3.head()

,name,latitud,longitud,geometry
0,Regal E-Walk 4DX & RPX,40.756960,-73.989286,POINT (-73.98929 40.75696)
1,Zaro's Bakery,40.752875,-73.976946,POINT (-73.97695 40.75288)
2,Boomer Esiason's Stadium Grill,40.757840,-73.987099,POINT (-73.98710 40.75784)
3,Stadium Grill At Bowlmor Lanes,40.757211,-73.987395,POINT (-73.98739 40.75721)


In [68]:
Map(Layer(gdf3, popup_hover = [popup_element("name","Basketball Stadium")]))

In [69]:
office_location = [office_lat, office_long]
marker_office = Marker(location = office_location, icon = icono)
marker_office.add_to(map_1)
map_1

### Night clubs

In [70]:
url4 = 'https://api.foursquare.com/v2/venues/explore'
night_clubs = "4bf58dd8d48988d11f941735"

In [71]:
parametros_night_clubs = {"client_id" : tok1,
              "client_secret" : tok2,
              "v": "20180323",
              "ll": f"{nyc.get('coordinates')[0]},{nyc.get('coordinates')[1]}",
              "query":f"night_clubs","radius":500
}

resp4 = requests.get(url = url_query, params = parametros_night_clubs)
data4 = json.loads(resp4.text)

In [72]:
data4.keys()

dict_keys(['meta', 'response'])

In [73]:
decoding_data4 = data4.get("response")

In [74]:
for k,v in decoding_data4.items():
    print(k,v,"\n")

headerLocation Rockefeller Center 

headerFullLocation Rockefeller Center, New York 

headerLocationGranularity neighborhood 

query night_clubs 

totalResults 5 

suggestedBounds {'ne': {'lat': 40.7637189045, 'lng': -73.97242357822688}, 'sw': {'lat': 40.7547188955, 'lng': -73.98428322177313}} 

groups [{'type': 'Recommended Places', 'name': 'recommended', 'items': [{'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '52f2aa6f498ec156f4824876', 'name': 'The Tonight Show starring Jimmy Fallon', 'contact': {}, 'location': {'address': '30 Rockefeller Plz', 'lat': 40.759361587972265, 'lng': -73.97980660674304, 'labeledLatLngs': [{'label': 'display', 'lat': 40.759361587972265, 'lng': -73.97980660674304}], 'distance': 123, 'postalCode': '10112', 'cc': 'US', 'city': 'New York', 'state': 'NY', 'country': 'United States', 'formattedAddress': ['30 Rockefeller Plz', 'New York, NY 10112', 'United Stat

In [75]:
decoded4 = decoding_data4.get("groups")[0]

In [76]:
night_clubs = decoded4.get("items")

In [77]:
map_night_clubs = ["venue","name"]

In [78]:
m_latitud_nightclubs = ["venue","location","lat"]
m_longitud_nightclubbs = ["venue","location","lng"]

In [79]:
def getFromDict(diccionario4,mapa4):
    return reduce (operator.getitem,mapa4,diccionario4)

In [80]:
print(getFromDict(night_clubs[0],map_night_clubs))

The Tonight Show starring Jimmy Fallon


In [81]:
night_clubs_nyc = []
for dic in night_clubs:
    paralista4 = {}
    paralista4["name"] = getFromDict(dic,map_night_clubs)
    paralista4["latitud"] = getFromDict(dic,m_latitud_nightclubs)
    paralista4["longitud"] = getFromDict(dic,m_longitud_nightclubbs)
    night_clubs_nyc.append(paralista4)

In [82]:
night_clubs_nyc[0]
dataframefina4 = pd.DataFrame(night_clubs_nyc)
dataframefina4.head()

,name,latitud,longitud
0,The Tonight Show starring Jimmy Fallon,40.759362,-73.979807
1,Starry Night by Vincent van Gogh,40.761575,-73.977106
2,New York Sports Clubs,40.759151,-73.981686
3,Night Hotel Times Square,40.759131,-73.983739
4,Saturday Night Live Standby Line,40.758811,-73.979580


In [83]:
gdf4 = gpd.GeoDataFrame(dataframefina4, geometry = gpd.points_from_xy(dataframefina4.longitud,dataframefina4.latitud))
gdf4.head()

,name,latitud,longitud,geometry
0,The Tonight Show starring Jimmy Fallon,40.759362,-73.979807,POINT (-73.97981 40.75936)
1,Starry Night by Vincent van Gogh,40.761575,-73.977106,POINT (-73.97711 40.76157)
2,New York Sports Clubs,40.759151,-73.981686,POINT (-73.98169 40.75915)
3,Night Hotel Times Square,40.759131,-73.983739,POINT (-73.98374 40.75913)
4,Saturday Night Live Standby Line,40.758811,-73.979580,POINT (-73.97958 40.75881)


In [84]:
Map(Layer(gdf4, popup_hover = [popup_element("name","Night clubs")]))

### 5

In [117]:
url5 = 'https://api.foursquare.com/v2/venues/explore'
travel_transport = "4d4b7105d754a06379d81259"

In [118]:
parametros_travel = {"client_id" : tok1,
              "client_secret" : tok2,
              "v": "20180323",
              "ll": f"{nyc.get('coordinates')[0]},{nyc.get('coordinates')[1]}",
              "query":f"travel_transport",
                ## "radius":1000
                    }

In [119]:
resp5 = requests.get(url = url_query, params = parametros_travel)
data5 = json.loads(resp5.text)

In [120]:
data5.keys()

dict_keys(['meta', 'response'])

In [121]:
decoding_data5 = data5.get("response")
for k,v in decoding_data5.items():
    print(k,v,"\n")

suggestedRadius 1800 

headerLocation Rockefeller Center 

headerFullLocation Rockefeller Center, New York 

headerLocationGranularity neighborhood 

query travel_transport 

totalResults 9 

suggestedBounds {'ne': {'lat': 40.86118906444044, 'lng': -73.92832985}, 'sw': {'lat': 40.664859646750884, 'lng': -74.07324715}} 

groups [{'type': 'Recommended Places', 'name': 'recommended', 'items': [{'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '4accc499f964a52070c920e3', 'name': 'Theater Row - The Acorn', 'contact': {}, 'location': {'address': '410 W 42nd St', 'crossStreet': 'btw 9th and 10th Ave', 'lat': 40.7585514688034, 'lng': -73.9934189068339, 'labeledLatLngs': [{'label': 'display', 'lat': 40.7585514688034, 'lng': -73.9934189068339}], 'distance': 1272, 'postalCode': '10036', 'cc': 'US', 'city': 'New York', 'state': 'NY', 'country': 'United States', 'formattedAddress': ['410 W 42nd St (b

In [122]:
decoded5 = decoding_data5.get("groups")[0]

In [123]:
transport = decoded5.get("items")

map_transport = ["venue","name"]

In [124]:
m_latitud_transport = ["venue","location","lat"]
m_longitud_transport = ["venue","location","lng"]
def getFromDict(diccionario5,mapa5):
    return reduce (operator.getitem,mapa5,diccionario5)

In [125]:
print(getFromDict(transport[0],map_transport))

Theater Row - The Acorn


In [126]:
transport_nyc = []
for dic in transport:
    paralista5 = {}
    paralista5["name"] = getFromDict(dic,map_stadium)
    paralista5["latitud"] = getFromDict(dic,m_latitud_stadium)
    paralista5["longitud"] = getFromDict(dic,m_longitud_stadium)
    transport_nyc.append(paralista5)

In [128]:
transport_nyc[0]
dataframefina5 = pd.DataFrame(transport_nyc)
dataframefina5.head()

gdf5 = gpd.GeoDataFrame(dataframefina5, geometry = gpd.points_from_xy(dataframefina5.longitud,dataframefina5.latitud))
gdf5.head()

Map(Layer(gdf5, popup_hover = [popup_element("name","Transports")]))